# Подключение библиотек и загрузка данных

In [1]:
import pandas as pd
from datetime import datetime
import grequests
from bs4 import BeautifulSoup as BS
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('main_task_new.csv')

# Вспомогательные функции

In [2]:
def str_to_array(string):
    # Выделение из строки (кухни и отзывы) массива
    string = string.replace(']', '').replace('[', '').replace('\'', '')
    arr = string.split(', ')
    return arr


def str_split_arrays(string):
    # Получение двух массивов из отзывов
    arr1, arr2 = string.split('], [')
    return str_to_array(arr1), str_to_array(arr2)


def get_max_date(arr):
    # Получаем максимальную дату из массива строк
    arr = [datetime.strptime(d, '%m/%d/%Y').date() for d in arr if d]
    if arr:
        return max(arr)
    return None

# Что из себя представляют данные

In [3]:
df

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963
...,...,...,...,...,...,...,...,...,...,...
39995,id_499,Milan,"['Italian', 'Vegetarian Friendly', 'Vegan Opti...",500.0,4.5,$$ - $$$,79.0,"[['The real Italian experience!', 'Wonderful f...",/Restaurant_Review-g187849-d2104414-Reviews-Ro...,d2104414
39996,id_6340,Paris,"['French', 'American', 'Bar', 'European', 'Veg...",6341.0,3.5,$$ - $$$,542.0,"[['Parisian atmosphere', 'Bit pricey but inter...",/Restaurant_Review-g187147-d1800036-Reviews-La...,d1800036
39997,id_1649,Stockholm,"['Japanese', 'Sushi']",1652.0,4.5,NaN,4.0,"[['Good by swedish standards', 'A hidden jewel...",/Restaurant_Review-g189852-d947615-Reviews-Sus...,d947615
39998,id_640,Warsaw,"['Polish', 'European', 'Eastern European', 'Ce...",641.0,4.0,$$ - $$$,70.0,"[['Underground restaurant', 'Oldest Restaurant...",/Restaurant_Review-g274856-d1100838-Reviews-Ho...,d1100838


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine Style      30717 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        26114 non-null  object 
 6   Number of Reviews  37457 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


In [5]:
print(df['Restaurant_id'].value_counts()[:10])
for i in df['Restaurant_id'].value_counts()[:3].index:
    print(i, df[df['Restaurant_id'] == i].City.value_counts())

id_633    18
id_436    18
id_227    18
id_871    18
id_430    17
id_585    17
id_71     17
id_534    17
id_321    17
id_344    17
Name: Restaurant_id, dtype: int64
id_633 Milan         1
Budapest      1
Prague        1
Geneva        1
Copenhagen    1
Oslo          1
Athens        1
Helsinki      1
Vienna        1
Madrid        1
Paris         1
Krakow        1
Hamburg       1
Lyon          1
Zurich        1
Munich        1
Rome          1
Stockholm     1
Name: City, dtype: int64
id_436 Barcelona     1
Copenhagen    1
Vienna        1
Hamburg       1
Athens        1
Munich        1
Rome          1
London        1
Berlin        1
Stockholm     1
Krakow        1
Ljubljana     1
Milan         1
Oslo          1
Luxembourg    1
Budapest      1
Oporto        1
Madrid        1
Name: City, dtype: int64
id_227 Luxembourg    1
Bratislava    1
Zurich        1
Geneva        1
Vienna        1
Milan         1
Budapest      1
Barcelona     1
Oslo          1
Helsinki      1
Ljubljana     1
Dublin       

# Добавление новых признаков

In [6]:
# Рестораны с одинаковым id - принадлежат одной сети (так как есть рестораны с одним и тем же ID, но в разных городах)
duplicated_dict = dict(df['Restaurant_id'].value_counts())
df['restaurants_in_network'] = df['Restaurant_id'].apply(lambda x: duplicated_dict[x])

# Количество кухонь (1 если данные отсутствуют)
df['Cuisine Count'] = df['Cuisine Style'].apply(lambda x: len(str_to_array(x)) if isinstance(x, str) else 1)

# данные о последнем отзыве и перерыве между отзывами
df['last_review_date'] = df['Reviews'].apply(lambda x: get_max_date(str_split_arrays(x)[1]))
last_review_date = df.sort_values('last_review_date', ascending=False).iloc[0]['last_review_date']
df['days_after_last_review'] = df['last_review_date'].apply(lambda x: (last_review_date - x).days if x else None)
df['days_between_reviews'] = df['Reviews'].apply(
    lambda x: abs((
        datetime.strptime(str_split_arrays(x)[1][0], '%m/%d/%Y').date() 
        -
        datetime.strptime(str_split_arrays(x)[1][1], '%m/%d/%Y').date()
    ).days) if (
        len(str_split_arrays(x)[1])==2 and str_split_arrays(x)[1][0] and str_split_arrays(x)[1][1]
    ) else None
)

# Добавим информацию с сайта tripadvisor

In [7]:
parsed_file = open("parsed.txt","r")
lines = parsed_file.readlines()
parsed_data = {}

for l in lines:
    k, v = l.replace('\n','').split(':')
    parsed_data[k] = v
    
parsed_file.close()

LOADING = False  # флаг для реальной загрузки/обновления данных.

if LOADING:
    def get_price_range_data(link):
        url = 'https://www.tripadvisor.com' + link
        with grequests.Session() as session:
            session.headers['Accept'] = 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8'
            session.headers['User-Agent'] = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 ' \
                                            '(KHTML, like Gecko) Chrome/91.0.4472.164 Safari/537.36'
            price_range = 'None'
            try:
                resp = session.get(url, timeout=5)
                soup = BS(resp.content, 'html.parser')
                _range = soup.find(class_='drUyy').contents[0]
                if '$' in _range:
                    price_range = _range
            except:
                price_range = 'Exception'

        return price_range

    for i, row in df[ df['Price Range'].isna() ].iterrows():
        rest_id = row['ID_TA']
        if rest_id not in parsed_data:
            price_range = get_price_range_data(row['URL_TA'])
            line = f'{rest_id}:{price_range}'
            parsed_data[rest_id] = price_range
            parsed_file = open("parsed.txt","a")
            parsed_file.write(f'{line}\n')
            parsed_file.close()

def set_price_range_file(row):
    if isinstance(row['Price Range'], str) and '$' in row['Price Range']:
        return row['Price Range']

    # Используем price range, если он указан у trip advisor
    loaded_range = parsed_data.get(row['ID_TA'], '')
    if '$' in loaded_range:
        return loaded_range

    return row['Price Range']

df['Price Range'] = df.apply(set_price_range_file, axis = 1)

# Заполнение пропусков и подготовка данных для модели

In [8]:
# Установим пропущенный price range 

def set_price_range_network(row):
    if isinstance(row['Price Range'], str) and '$' in row['Price Range']:
        return row['Price Range']

    # Заполним price range, если сеть установила для всех остальных своих ресторанов одинаковый ценовой диапазон.
    same_rest = df[ ( df['Price Range'].notna() ) & ( df['Restaurant_id'] == row['Restaurant_id'] ) ]
    if len(same_rest['Price Range'].unique()) == 1:
        return same_rest.iloc[0]['Price Range']
    
    return row['Price Range']

df['Price Range'] = df.apply(set_price_range_network, axis = 1)

In [9]:
# переведём price range в цифровой вид
price_range_dict = {
    '$': 10,
    '$$ - $$$': 25,
    '$$$$': 40,
}

df['price_range'] = df['Price Range'].apply(lambda x: price_range_dict.get(x))

In [10]:
def fill_by_city_mean(row):
    global changed, keeped
    if not pd.isna(row['price_range']):
        return row['price_range']

    # Заполним ценовой диапазон, беря среднее количество по городу
    val = df[ ( df['price_range'].notna() ) & ( df['City'] == row['City'] ) ]['price_range'].mean()
    if not pd.isna(val):
        return val

df['price_range'] = df.apply(fill_by_city_mean, axis = 1)

In [11]:
# Добавим новые признаки по имеющимся городам

df = pd.get_dummies(df, columns=['City',])

In [12]:
# Убираем ненужные признаки и заполняем оставшиеся пропуски

df.drop(['Cuisine Style', 'Price Range', 'Reviews', 'URL_TA', 'ID_TA', 'last_review_date'], axis = 1, inplace=True)

df = df.fillna(0)

In [13]:
df

,Restaurant_id,Ranking,Rating,Number of Reviews,restaurants_in_network,Cuisine Count,days_after_last_review,days_between_reviews,price_range,City_Amsterdam,...,City_Munich,City_Oporto,City_Oslo,City_Paris,City_Prague,City_Rome,City_Stockholm,City_Vienna,City_Warsaw,City_Zurich
0,id_5569,5570.0,3.5,194.0,3,3,57.0,41.0,25.00000,0,...,0,0,0,1,0,0,0,0,0,0
1,id_1535,1537.0,4.0,10.0,10,1,235.0,382.0,25.00000,0,...,0,0,0,0,0,0,1,0,0,0
2,id_352,353.0,4.5,688.0,8,7,49.0,2.0,40.00000,0,...,0,0,0,0,0,0,0,0,0,0
3,id_3456,3458.0,5.0,3.0,4,1,0.0,0.0,25.00000,0,...,0,0,0,0,0,0,0,0,0,0
4,id_615,621.0,4.0,84.0,14,3,100.0,272.0,25.00000,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,id_499,500.0,4.5,79.0,14,4,72.0,34.0,25.00000,0,...,0,0,0,0,0,0,0,0,0,0
39996,id_6340,6341.0,3.5,542.0,2,5,67.0,9.0,25.00000,0,...,0,0,0,1,0,0,0,0,0,0
39997,id_1649,1652.0,4.5,4.0,10,2,480.0,3127.0,23.69783,0,...,0,0,0,0,0,0,1,0,0,0
39998,id_640,641.0,4.0,70.0,8,5,230.0,23.0,25.00000,0,...,0,0,0,0,0,0,0,0,1,0


# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [14]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['Restaurant_id', 'Rating'], axis = 1)
y = df['Rating']

In [15]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [16]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [17]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [18]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [19]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.21615299999999998
